## Dietary Food Recomender System

In [39]:
# Import Library
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
# Load Dataset
df = pd.read_csv('after_encoding.csv')
df.head()

C:\Users\jason\AppData\Local\Temp\ipykernel_17696\156284050.py:2: DtypeWarning: Columns (7,117,1309,1340,1494,1964,2306,2330,2628,2799,2800,2816,3330,3605,3869,4069,4071,4073,4214,4233,4238,4279,4501,4724,4727,4797,4806,4816,4875,5441,5477,5483,5566,6024,6077,6079,6091,6104,6125,6177,6278,6632,6640,6950,6953,6966,6969,6991,6996,7223,7243,7304,7333,7389,8018,8106,9213) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('after_encoding.csv')


,name,ingredients,steps,calories,carbs,protein,fat,servings,image_url,link,...,"zucchini, cut into small chunks","zucchini, cut into spirals using a spiral slicer","zucchini, diced","zucchini, grated and squeezed dry","zucchini, peeled and finely chopped","zucchini, shredded","zucchini, sliced in half vertically, then sliced crosswise","zucchini, thinly sliced",zucchinis,"zucchinis, thinly sliced lengthwise"
0,air fryer lemon garlic parmesan chicken,"[{'quantity': '1 1/2', 'unit': 'pounds', 'name...",Gather all ingredients.. Preheat an air fryer ...,365,8g,0,17g,4,https://www.allrecipes.com/thmb/5nJvgXENSeFx82...,https://www.allrecipes.com/air-fryer-lemon-gar...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
1,air fryer s’mores,"[{'quantity': '1', 'unit': 'sleeve', 'name': '...",Preheat an air fryer to 380 degrees F (193 deg...,143,20g,0,6g,10,https://www.allrecipes.com/thmb/_EDaiFt0gIGQOL...,https://www.allrecipes.com/air-fryer-s-mores-r...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
2,air fryer baked yams,"[{'quantity': '1', 'unit': 'pieces', 'name': '...",Preheat an air fryer to 400 degrees F (200 deg...,283,62g,0,3g,1,https://www.allrecipes.com/thmb/156WNgRfzvGn-s...,https://www.allrecipes.com/air-fryer-baked-yam...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
3,lemon garlic butter chicken spiedini,"[{'quantity': '1/2', 'unit': 'cup', 'name': 'e...","Whisk together olive oil, wine, 2 tablespoons ...",636,21g,0,41g,6,https://cdn.jwplayer.com/v2/media/ahbYMLcr/thu...,https://www.allrecipes.com/lemon-garlic-butter...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
4,air fryer grilled pimento cheese,"[{'quantity': '4', 'unit': 'slices', 'name': '...",Preheat the air fryer to 370 degrees F (188 de...,902,108g,0,40g,2,https://www.allrecipes.com/thmb/cdL3DKZH3beUk5...,https://www.allrecipes.com/air-fryer-grilled-p...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0


In [41]:
# Select columns with float64 and int64 types
numeric_df = df.select_dtypes(include=['float64', 'int64'])
numeric_df = numeric_df.drop(columns=['calories', 'protein'])
# Display the result
print(numeric_df)

      (1 1/4-inch-thick) tenderloin steaks  \
0                                        0   
1                                        0   
2                                        0   
3                                        0   
4                                        0   
...                                    ...   
2905                                     0   
2906                                     0   
2907                                     0   
2908                                     0   
2909                                     0   

      (1 inch thick) filet mignon steaks  \
0                                      0   
1                                      0   
2                                      0   
3                                      0   
4                                      0   
...                                  ...   
2905                                   0   
2906                                   0   
2907                                   0   
2908   

In [42]:
# Select All the Ingredients Columns
ingredients = numeric_df

# Compute Cosine Similarity Matrix
similarity = cosine_similarity(ingredients)

In [43]:
# Base Recommendation Function
def recommend_similar_recipes(name, filter_func=None, top_n=10):
    # Check if the Recipe Exists
    if name not in df['name'].values:
        return f"Recipe '{name}' not found."
    
    # Get the Index of the Recipe
    recipe_id = df[df['name'] == name].index[0]
    
    # Get Similarity Score for the Given Recipe
    sim_scores = list(enumerate(similarity[recipe_id]))
    
    # Sort Scores in Descending Order of Similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Exclude the recipe itself
    sim_scores = sim_scores[1:]
    
    # Apply Calorie Filter if Provided
    if filter_func:
        sim_scores = [score for score in sim_scores if filter_func(score[0])]
        
    # Get the top_n Indices and Scores after Filtering
    sim_scores = sim_scores[:top_n]
    recipe_indices = [i[0] for i in sim_scores]
    recipe_similarities = [i[1] for i in sim_scores]
    
    # Prepare the result as a list of tuples: (recipe_name, similarity_score)
    recommended_recipes = [
        (df['name'].iloc[idx], similarity) 
        for idx, similarity in zip(recipe_indices, recipe_similarities)
    ]
    
    return recommended_recipes

In [44]:
# Recommendation Function for Higher Recommended Recipes
def recommend_higher_calorie_recipe(name, top_n=10):
    target_calorie = df.loc[df['name'] == name, 'calories'].values[0]
    return recommend_similar_recipes(
        name,
        filter_func=lambda idx: df.iloc[idx, df.columns.get_loc('calories')] > target_calorie,
        top_n=top_n
    )

# Recommendation Function for Lower Recommended Recipes
def recommend_lower_calorie_recipe(name, top_n=10):
    target_calorie = df.loc[df['name'] == name, 'calories'].values[0]
    return recommend_similar_recipes(
        name,
        filter_func=lambda idx: df.iloc[idx, df.columns.get_loc('calories')] < target_calorie,
        top_n=top_n
    )

In [49]:
# Test the recommender systems
print("Similar recipes with higher calories:")
print(recommend_higher_calorie_recipe('roscoe\'s korean fried chicken', top_n=5))

print("\nSimilar recipes with lower calories:")
print(recommend_lower_calorie_recipe('roscoe\'s korean fried chicken', top_n=5))

Similar recipes with higher calories:
[('southern style chicken and dumplings', 0.6179712427110777), ("d' best chicken n' dumplings", 0.5561794544861245), ('fried apple pies', 0.5511962472195403), ('caramel cake', 0.40674474901953284), ("brad's slow cooker chicken parmesan", 0.32972737279545977)]

Similar recipes with lower calories:
[('polish chicken and dumplings', 0.6578126706026299), ('prosciutto plum focaccia', 0.6493157510909111), ('sweet dark cherry cobbler', 0.6089015758016155), ('southern chicken and dumplings', 0.6059361052866542), ('perfect pan pizza', 0.6055760980577498)]


In [50]:
# Locate the rows of the specific recipes by their names
recipes_to_check = ['polish chicken and dumplings','prosciutto plum focaccia', 'sweet dark cherry cobbler', 'southern chicken and dumplings','perfect pan pizza', 'roscoe\'s korean fried chicken']
selected_recipes = df.loc[df['name'].isin(recipes_to_check)]

In [51]:
# Display the selected rows
print("\nSelected calories:")
df_selected_recipes = pd.DataFrame(selected_recipes)
df_selected_recipes


Selected high calories:


,name,ingredients,steps,calories,carbs,protein,fat,servings,image_url,link,...,"zucchini, cut into small chunks","zucchini, cut into spirals using a spiral slicer","zucchini, diced","zucchini, grated and squeezed dry","zucchini, peeled and finely chopped","zucchini, shredded","zucchini, sliced in half vertically, then sliced crosswise","zucchini, thinly sliced",zucchinis,"zucchinis, thinly sliced lengthwise"
202,roscoe's korean fried chicken,"[{'quantity': '12', 'unit': 'wingette or drumm...",Place a wire rack inside a rimmed baking sheet...,750,70g,0,41g,6,https://cdn.jwplayer.com/v2/media/vsnpJLMn/thu...,https://www.allrecipes.com/roscoes-korean-frie...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
913,prosciutto plum focaccia,"[{'quantity': '1', 'unit': 'packet', 'name': '...",Stir yeast and warm water together in a large ...,171,26g,0,5g,12,https://cdn.jwplayer.com/v2/media/BK1S8BIa/thu...,https://www.allrecipes.com/prosciutto-plum-foc...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
917,perfect pan pizza,"[{'quantity': '2 1/2', 'unit': 'cups', 'name':...","Whisk flour, yeast, and salt together in a lar...",562,66g,0,21g,4,https://cdn.jwplayer.com/v2/media/adGcaCWS/thu...,https://www.allrecipes.com/perfect-pan-pizza-r...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
2036,sweet dark cherry cobbler,"[{'quantity': '½', 'unit': 'cup', 'name': 'whi...",Preheat the oven to 400 degrees F (200 degrees...,220,38g,0,7g,8,https://www.allrecipes.com/thmb/7_FBBs_xoDWuUk...,https://www.allrecipes.com/recipe/234300/sweet...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
2106,polish chicken and dumplings,"[{'quantity': '1', 'unit': '(3 pound)', 'name'...","Place chicken, onion, and celery in a large po...",704,53g,0,34g,8,https://www.allrecipes.com/thmb/74Ihv_Mau1KUHm...,https://www.allrecipes.com/recipe/47705/polish...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0
2111,southern chicken and dumplings,"[{'quantity': '2', 'unit': 'tablespoons', 'nam...",Make the chicken: Melt butter in a large pot o...,616,42g,0,32g,8,https://www.allrecipes.com/thmb/gH1jcapzc4AaDy...,https://www.allrecipes.com/recipe/218560/south...,...,0,0,0.0,0.0,0,0.0,0,0.0,0,0


In [ ]:
# Locate the row with the given recipe name
recipe_name = "bulgogi (korean bbq)"
recipe_row = df.loc[df['name'] == recipe_name]

# Extract the link (assuming 'link' column exists)
recipe_link = recipe_row['link'].values[0]

# Display the link
print("Link for the recipe:", recipe_link)